# Data Augmentation
KoGPT2 모델을 통해 새로운 문장을 생성함으로써 data augmentation을 해줌

KoGPT2 는 앞 뒤 단어의 유사도를 통해 문장을 생성

## Load DataSet

In [ ]:
## 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## generate할 데이터 불러오기
!gdown 1Mo5cd2QMY4Obtj4hsRreSUfYetNAgQ7k

Downloading...
From: https://drive.google.com/uc?id=1Mo5cd2QMY4Obtj4hsRreSUfYetNAgQ7k
To: /content/NegDataSet_0706.csv
100% 705k/705k [00:00<00:00, 148MB/s]


## Preprocess Data

In [ ]:
import pandas as pd
df = pd.read_csv('NegDataSet_0706.csv')
df.dropna(inplace=True) # 결측치 제거
df

,comment,star
0,산지 하루만에 고장났어요 배터리가 다된거면 이해한다 싶어서 자주가던 세운스퀘어 시계...,1
1,색상도 디자인도 예쁜데 시계가 작동을 안하네요 밧데리가 없나봐요,1
2,깔끔하고 마음에 들어요 잘쓸게요,1
3,ㅇ 뻐요 가격좋고 세계상이 괜찮은듯해요,1
4,친구선물했는데 크게 했더니 양말신고 발목 잘 맞고 길이만 좀 길다 하더라구요 예쁘다...,1
...,...,...
4986,연보라색을 생각하고 샀는데 진보라색입니다 색이 실망스러웠고 목부분이 너무 커서 고정...,2
4987,기모가 아쥬그냥 끝나버리네요 바지랑 티랑 세탁하면 다른옷에 기모 생겨요 다들 단독 ...,2
4988,고무마개 넘 안빠져서 공구로 열었어요 엎을까봐 무서웠네요,2
4989,일단 옷은 예뻐요 후드티니까 어디에나 잘 어울리고요 그런데 옷에 뭐가 묻고 머리카락...,2


In [ ]:
# star = 2 이외의 데이터 삭제
# star data의 편향으로 2 데이터만 generate
delet = df[df['star'].apply(str).str.contains('2', na=False)==False].index
df.drop(delet, inplace = True)
df

,comment,star
5,빠른 배송에 깔끔한 제품으로 선물 받은 사람도 대만족입니다,2
27,한달사용기길이가 긴 레인부츠 신다가 좀 더 간편하게 신을 수 있는 걸 찾아서 구매했...,2
45,한달사용기비가 솔직히 완벽하게 안들어오진 않지만 그래도 덜 장화같고 좋아요 대신 오...,2
57,너무예뻐요 ㅎㅎ난쟁이 똥자룽라서 짤븡ㄴ거 찾아가 이거사요 넘마에 드네용,2
74,신는데 삿더니 딱맞아요 발볼좁은편인데 발볼도 딱맞아요 안쪽에 쓸리는부분이잇어서 긴...,2
...,...,...
374703,연보라색을 생각하고 샀는데 진보라색입니다 색이 실망스러웠고 목부분이 너무 커서 고정...,2
376132,기모가 아쥬그냥 끝나버리네요 바지랑 티랑 세탁하면 다른옷에 기모 생겨요 다들 단독 ...,2
376604,고무마개 넘 안빠져서 공구로 열었어요 엎을까봐 무서웠네요,2
377112,일단 옷은 예뻐요 후드티니까 어디에나 잘 어울리고요 그런데 옷에 뭐가 묻고 머리카락...,2


In [ ]:
## comment 최대길이와 평균길이 확인
import numpy as np
t_len = np.array([len(comment) for comment in list(df['comment'])])

max(t_len), t_len.mean()

(1461, 54.3021438589461)

##Fine tuning

In [ ]:
## transformer install
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 15.1 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 101 kB 14.8 MB/s 
     |████████████████████████████████| 6.6 MB 60.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
## datasets 패키지 설치
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 12.3 MB/s 
     |████████████████████████████████| 140 kB 70.7 MB/s 
     |████████████████████████████████| 212 kB 71.3 MB/s 
     |████████████████████████████████| 1.1 MB 62.2 MB/s 
     |████████████████████████████████| 127 kB 28.1 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 144 kB 64.9 MB/s 
     |████████████████████████████████| 271 kB 73.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
## sentencepiece 패키지 설치
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 13.3 MB/s 


In [ ]:
## 필요한 Library import
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_metric, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, DataCollatorWithPadding

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
## train:test = 8:2 로 나눠줌
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
model_name = "skt/kogpt2-base-v2"

In [ ]:
## 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name,
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>')

# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
#     bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#     pad_token='<pad>', mask_token='<mask>')

# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
## 모델설정
model = AutoModelForCausalLM.from_pretrained(model_name)

# model = GPT2LMHeadModel.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(path + '/results/checkpoint-3991/') ## checkpoint 불러와서 학습재개

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [ ]:
# 데이터셋 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'comment'],
        num_rows: 27880
    })
    test: Dataset({
        features: ['index', 'comment'],
        num_rows: 6971
    })
})

In [ ]:
## 전처리함수
def preprocess_function(examples):
    return tokenizer(examples["comment"], truncation=True, padding='max_length', max_length=256) # 최대길이 256까지 패딩
    # return tokenizer([" ".join(x) for x in examples["content"]], truncation=True, padding='max_length', max_length=256)

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4, # 다중처리 활성화, 토크나이저 성능 향상
    remove_columns=dataset["train"].column_names,
)

Parameter 'function'=<function preprocess_function at 0x7f18e8d93a70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/7 [00:00<?, ?ba/s]

#1:   0%|          | 0/7 [00:00<?, ?ba/s]

#2:   0%|          | 0/7 [00:00<?, ?ba/s]

#3:   0%|          | 0/7 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 27880
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 6971
    })
})

In [ ]:
enc = tokenized_dataset['train']['input_ids'][0]
tokenizer.decode(enc)

'가죽인데 가벼워요. 깔끔해서 어디에나 어울리네요.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
block_size = 256

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

#0:   0%|          | 0/7 [00:00<?, ?ba/s]

#1:   0%|          | 0/7 [00:00<?, ?ba/s]

#2:   0%|          | 0/7 [00:00<?, ?ba/s]

#3:   0%|          | 0/7 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 27880
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6971
    })
})

In [ ]:
enc = lm_dataset['train']['input_ids'][0]
tokenizer.decode(enc)

'가죽인데 가벼워요. 깔끔해서 어디에나 어울리네요.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
path = '/content/drive/MyDrive/Generation'

training_args = TrainingArguments(
    output_dir=path+"/results",
    save_strategy="epoch",
    save_total_limit=3,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    num_train_epochs=6.0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

In [ ]:
trainer.train('/content/drive/MyDrive/Generation/results/checkpoint-17425') # ()안에 checkpoint를 넣어 학습을 이어서 할 수 있음

Loading model from /content/drive/MyDrive/Generation/results/checkpoint-17425.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27880
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 20910
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 5
  Continuing training from global step 17425
  Will skip the first 5 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data 

0it [00:00, ?it/s]

Epoch,Training Loss,Validation Loss
6,2.675000,2.498757


***** Running Evaluation *****
  Num examples = 6971
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/Generation/results/checkpoint-20910
Configuration saved in /content/drive/MyDrive/Generation/results/checkpoint-20910/config.json
Model weights saved in /content/drive/MyDrive/Generation/results/checkpoint-20910/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/Generation/results/checkpoint-10455] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=20910, training_loss=0.44231710192234763, metrics={'train_runtime': 2050.6105, 'train_samples_per_second': 81.576, 'train_steps_per_second': 10.197, 'total_flos': 2.185446555648e+16, 'train_loss': 0.44231710192234763, 'epoch': 6.0})

### TEST

In [ ]:
## 테스트용
## 한 문장만 생성해봄
text = df['comment'][26]
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                          max_length=256,
                          repetition_penalty=2.0,
                          pad_token_id=tokenizer.pad_token_id,
                          eos_token_id=tokenizer.eos_token_id,
                          bos_token_id=tokenizer.bos_token_id,
                          use_cache=True)
generated = tokenizer.decode(gen_ids[0])
generated = generated[128:]
print(generated)

Input length of input_ids is 278, but ``max_length`` is set to 256. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


는 비 그치고 나면 레인부츠 너무 무겁고 더워서 너무 불편했는데 이건 멋내기 부츠 같아서 길지 않아 그런 불편함이 없네요. 일단 부츠임에도 무겁지 않습니다. 레인부츠 특유의 종아리 퉁퉁 이런 느낌도 없구요 천으로 된 부분에 저는 개인적으로 방수스프레이 뿌렸어요. 다이소에서 저렴하게 살 수 있습니다. 천으로 된 부분 때문에 물이 들어올까봐 고민되시는 분께는 방수스프레이 추천드려요. 헌터랑 고민하닥 가격메리트가 말도안되게 너무 좋구 디자인이 스타일리시해서 이걸로 결정했어요. 깔창도 메모리폼같은? 느낌이어서 상당히 편합니다. 제가 발바닥이 매우 얇고 굉장히 칼발이라 피로감이 심한데 부츠임에도 피로감 심하지 않았어요. 눈 올때 울양말 신고 신기위해 일부러 한사이즈 업해서 구매했습니다. 맨발로 신으면 조금 크긴 한데 불편할 정도 아니에요. 꼭 구매하세요 너무 예뻐요 더보기 전에


### 학습시킨 모델, tokenizer 저장

In [ ]:
tokenizer.save_pretrained(path + '/saved')
model.save_pretrained(path + '/saved')

tokenizer config file saved in /content/drive/MyDrive/Generation/saved/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Generation/saved/special_tokens_map.json
Configuration saved in /content/drive/MyDrive/Generation/saved/config.json
Model weights saved in /content/drive/MyDrive/Generation/saved/pytorch_model.bin


### 저장한 모델, tokenizer 불러와서 *사용하기*

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/2. SUB _ MUSINSA/1. model/Generation'
tokenizer = AutoTokenizer.from_pretrained(path + '/saved')
model = AutoModelForCausalLM.from_pretrained(path + '/saved')

In [ ]:
df['comment'][27]

'편하게 아주 잘신고있어요 무난하게 이쁘네요'

In [ ]:
df

,comment,star
0,산지 하루만에 고장났어요 배터리가 다된거면 이해한다 싶어서 자주가던 세운스퀘어 시계...,1
1,색상도 디자인도 예쁜데 시계가 작동을 안하네요 밧데리가 없나봐요,1
2,깔끔하고 마음에 들어요 잘쓸게요,1
3,ㅇ 뻐요 가격좋고 세계상이 괜찮은듯해요,1
4,친구선물했는데 크게 했더니 양말신고 발목 잘 맞고 길이만 좀 길다 하더라구요 예쁘다...,1
...,...,...
4986,연보라색을 생각하고 샀는데 진보라색입니다 색이 실망스러웠고 목부분이 너무 커서 고정...,2
4987,기모가 아쥬그냥 끝나버리네요 바지랑 티랑 세탁하면 다른옷에 기모 생겨요 다들 단독 ...,2
4988,고무마개 넘 안빠져서 공구로 열었어요 엎을까봐 무서웠네요,2
4989,일단 옷은 예뻐요 후드티니까 어디에나 잘 어울리고요 그런데 옷에 뭐가 묻고 머리카락...,2


In [ ]:
## max_len이 mean_len에비해 너무 큼
## try, except를 사용하여 IndexError가 발생하면 넘기기

list_comment = list(df['comment'])
aug_df = {'comment':[]} # 빈 dict 생성
for text, max in zip(tqdm(list_comment), tqdm(t_len)):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    try:
      gen_ids = model.generate(input_ids,
                                max_length=max*2, # 각 문장길이의 2배를 max_len으로 설정, 각 문장 길이만큼 생성하기위해
                                repetition_penalty=2.0, # 반복 통제
                                pad_token_id=tokenizer.pad_token_id,
                                eos_token_id=tokenizer.eos_token_id,
                                bos_token_id=tokenizer.bos_token_id,
                                use_cache=True)
      generated = tokenizer.decode(gen_ids[0])
      generated = generated[max:]
      aug_df['comment'].append(generated) # generate한 문장을 aug_df라는 빈 dict에 append 
    except IndexError:
      pass

  0%|          | 0/4991 [00:00<?, ?it/s]

  0%|          | 0/4991 [00:00<?, ?it/s]

In [ ]:
aug_df

{'comment': ['! 가성비로는 최고인듯 합미당 사이즈도 딱맞고 잘 맞아여 근데 좀 얇아서 오래입을수 있을지는 모르겠지만 그래도 예뻐서 그냥 입으려구용. 디자인이나 마감처리는 만족합니에오ᄒ 더보기엔 너무 작았습니닷ᄏᄃᄉᄀᄎ 이쁘긴해<unk> ᄉᄇᄋᄐᄅ 감사함돠 번창하세욥 굿굿이예염 저는 에 인인데 샀어도 됐을뻔했음 암튼 좋앗숩시다 진짜 편하고 좋은것같아욤 많이파시고 다음에 또 이용할게욘 그럼 이만되죠 무신사 강추드립니가 됩시더 굳굳 여기서 바로사고싶내써옹 배송올때부터 기분조심해야겟슴 당첨되면 무조건 로얄탠덤으로 적립금주셔야될꺼임 그리고 제가 받은건 택배기사님이랑 통화했는데 친절하게 응대해주셨다고 하셧읍니까 그점만족할께유 다음부턴 조심좀 부탁드려영 앞으로 계속 애용하겠습ᄂ 요번부터는 믿고',
  ' 그래도 잘 쓰고 다닐게용ᄒᄉᄎ. 근데이런백이라서 더 좋습니다 감사합니당 무신사스탠디맨투멘토 자주 애용할께여 번창하시길 바랍니가닿ᄋᄆᄅ감사',
  ' 감사합니다. 더보기엔 좀 작아서 아쉽지만 그래도 예뻐서 자주 입을 것 같아용ᄒᄉ',
  '. 근데 생각보다 너무 얇아서 안에 뭐 껴입어야할거같아여 그래도 이쁘니까 잘입을게용',
  ' 근데 너무 얇아서 한겨울엔 못입을 것 같아용. 그래도 이뻐서 자주 신을것 같습니당ᄒᄉᄏ 사이즈도 딱 맞아욤 더보기에는 조금 크지만 만족합시다 감사해여 많이 파세욥 번창하십쇼 ᄉᄐᄎᄀᄇᄅᄋᄌᄂᄆᄑᄊᄈᄁ<unk>이댜오옹아',
  '! 편하게 잘 입을 것 같아용ᄒᄉᄐᄎᄏ 근데 좀 얇아서 요즘',
  '! 다만 아쉬운건 옷이 좀 무겁습니다. 그래도 이쁘니까 용서됩니당ᄒᄉᄋ 사이즈도 딱 맞고 좋았구여 많이 파세',
  ' 근데 너무이쁘네용ᄒᄉᄀᄎ 잘입겠습니다 감사합니당. 사이즈도 딱맞고 좋아여 많이파세욥 번창하셔유 무신사스탠드 자주 이용할게욤 굿굿 추천추천해염 배송빠름좋음오배송감사함더보기엔크지만예',
  '엔 좀 딱딱하지만 그래도 잘 신을게용ᄒ. 근데이런 페스티벌이라 저렴하게 구매해서 좋았습니당ᄏ 다른색상도 사고싶은 마음에 들어여 많이 파세염

In [ ]:
aug_df = pd.DataFrame(aug_df)
aug_df

,comment
0,! 가성비로는 최고인듯 합미당 사이즈도 딱맞고 잘 맞아여 근데 좀 얇아서 오래입을수...
1,그래도 잘 쓰고 다닐게용ᄒᄉᄎ. 근데이런백이라서 더 좋습니다 감사합니당 무신사스탠...
2,감사합니다. 더보기엔 좀 작아서 아쉽지만 그래도 예뻐서 자주 입을 것 같아용ᄒᄉ
3,. 근데 생각보다 너무 얇아서 안에 뭐 껴입어야할거같아여 그래도 이쁘니까 잘입을게용
4,근데 너무 얇아서 한겨울엔 못입을 것 같아용. 그래도 이뻐서 자주 신을것 같습니당...
...,...
4968,하지만 아쉬운점은 팔길이가 좀 길어서 접히는 부분이 있습니당. 그래도 이쁘긴합니까...
4969,! 그래도 이쁘긴해여. 근데 좀 많이 얇아오 그거빼곤 좋아용ᄒᄉᄋ 잘입을게욤 감사합...
4970,ᄏ 그래도 이뻐서 잘 입고 다닐게용ᄒᄉᄐᄎ 근데 좀 많이 얇아서 한겨울엔 못입을거...
4971,그래도 이 가격에 이런 옷을 산게 너무 기분좋네용ᄒᄉᄋ 감사합니당. 사이즈도 딱 ...


In [ ]:
# csv파일로 저장, 구분자 comma (,)
aug_df.to_csv('/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/2. SUB _ MUSINSA/4. csv/Generation_aug.csv',index=False)

In [ ]:
# tsv파일로 저장, 구분자 tab (\t)
aug_df.to_csv('/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/2. SUB _ MUSINSA/6. tsv/Generation_aug.tsv', sep='\t', encoding='utf-8',index=False)